In [ ]:
import pyspark
from pyspark.sql import SparkSession

# configSpark = pyspark.SparkConf().\
#                 setAll([('spark.executor.memory', '4g'),
#                         ('spark.executor.cores', '2'),
#                         ('spark.cores.max', '2'),
#                         ('spark.driver.memory', '4g')])

# construa a sessão no mesmo notebook que importou
spark = SparkSession\
    .builder\
    .appName("BlogPost")\
    .master("spark://DOUGPC.:7077")\
    .getOrCreate()

    # .master("spark://DOUGPC.:7077")\
    # .config(conf= configSpark)\

print(spark)

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [ ]:
#simples dataframe criado com pyspark
#print01
newDf = spark.range(1, 100)
newDf.head(10)
newDf.show()

In [ ]:
#print1.1
#criação de dataframe 
datafrm = spark.createDataFrame([('A', 20),\
                                ('B', 30),\
                                ('C', 40),\
                                ('D', 70),\
                                ('A', 30),\
                                ('C', 60)],
                                ['Prod', 'Valor'])
datafrm.show()

In [ ]:
#criação de um schema utilizando a linguagem Python.
from pyspark.sql.types import *

#atribuir à uma variável os schemas que irão para os dataframes
schemaFrame = StructType([StructField('Produto', StringType(), False),\
                            StructField('Categoria', StringType(), True),\
                            StructField('Valor', IntegerType() , False)])

#criando schema com Spark SQL.
schemaSQL = "Produto STRING, Categoria STRING, Valor INT"

#agora basta criar um novo dataframe e passar o schema com parâmetro da função

pythonFrame = spark.createDataFrame([('A', 'Alimento', 30),\
                                        ('B', 'Papelaria', 40),\
                                        ('C', 'Informatica', 30),\
                                        ('D', 'Papelaria', 40),\
                                        ('E', 'Alimento', 40),\
                                        ('F', 'Informatica', 100),\
                                        ('G', 'Papelaria', 80),\
                                        ('A', 'Alimento', 30),\
                                        ('D', 'Papelaria', 50),\
                                        ('C', 'Informatica', 50),\
                                        ('J', 'Bebida', 100)],
                                    ['Produto', 'Categoria', 'Valor'], schemaFrame)

frameSQL = spark.createDataFrame([('A', 'Alimento', 30),\
                                        ('B', 'Papelaria', 40),\
                                        ('C', 'Informatica', 30),\
                                        ('D', 'Papelaria', 40),\
                                        ('E', 'Alimento', 40),\
                                        ('F', 'Informatica', 100),\
                                        ('G', 'Papelaria', 80),\
                                        ('A', 'Alimento', 30),\
                                        ('D', 'Papelaria', 50),\
                                        ('C', 'Informatica', 50),\
                                        ('J', 'Bebida', 100)],
                                    ['Produto', 'Categoria', 'Valor'], schemaSQL)
pythonFrame.groupby('Categoria').sum('Valor').show()

#Long também pode ser considerado INT no Spark.
#ele escolhe qual datatype vai atribuir baseado no que for melhor para performance.
print(pythonFrame.printSchema())

In [ ]:
#quando tratamos de criar schema, O true or false diz respeito ao null
#se tal coluna do dataframe poderá ter valor null ou não, assim como uma tabela.

nullFrame = StructType([StructField('Produto', StringType(), False),\
                            StructField('Categoria', StringType(), True),\
                            StructField('Valor', FloatType(), False),\
                            StructField('Qntd', LongType(), False)])


pythonFrame = spark.createDataFrame([('A', 'Alimento', 30, 3),\
                                        ('B', 'Papelaria', 40,4 ),\
                                        ('C', 'Informatica', 30, 10),\
                                        ('D', 'Papelaria', 40, 5),\
                                        ('E', 'Alimento', 40, 3),\
                                        ('F', 'Informatica', 100, 10),\
                                        ('G', 'Papelaria', 80, 3),\
                                        ('A', 'Alimento', 30, 5),\
                                        ('D', 'Papelaria', 50,3),\
                                        ('C', 'Informatica', 50,5 ),\
                                        ('J', 'Bebida', 100, 10),\
                                        ('I','',30, 3)],
                                    ['Produto', 'Categoria', 'Valor', 'Qntd'], nullFrame)
print(pythonFrame.printSchema())
pythonFrame.collect()

In [ ]:
#criando um frame com toDF
#img03
rangeFrame = spark.range(500).toDF('rFrame')
rangeFrame.show()
rangeFrame.select(rangeFrame['rFrame'] + 10)

In [ ]:
#criando dataframes com uma lista de linhas

#IMG04
from datetime import date, datetime
import pandas as pd
from pyspark.sql import Row

rowListFrame = spark.createDataFrame([
    Row(a = 1, b = 2, c= 'String', d=date(2000, 1, 1)),
    Row(a = 2, b = 3, c= 'String 2', d= date(2000, 1, 2)),
    Row(a = 3, b = 4, c= 'String 3', d= date(2000, 1, 3))])

rowListFrame.show(1)

In [ ]:
#criando dataframe com schema explícito

#img 05
expSchemaFrame = spark.\
    createDataFrame([
        (1, 2., 'String 1', date(2000, 1, 1)),
        (2, 3., 'String 2', date(2000, 1, 2)),
        (3, 4., 'String 3', date(2000, 1, 3))
], schema= 'a int, b double, c string, d date')

spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

#se chamar uma função não funciona
#se utilizar print, funciona e exibe apenas uma linha
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 1)
print(expSchemaFrame)
expSchemaFrame.show()

In [ ]:
#exibindo o nome das colunas e informaçoes do dataframe
#img05.1
expSchemaFrame.columns

#para não passar as colunas, utilizo o atributo columns com o dataframe
#função describe para descrever o dataframe
expSchemaFrame.select(expSchemaFrame.columns).describe().show()

In [ ]:
#criando um dataframe a partir de um pandas dataframe
#utilizando um dicionário para cada coluna, passar suas linhas

#img06
pandasFrame = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': [date(2000, 1, 1), date(2000, 1, 2), date(2000, 1, 3)]
})

criapandasFrame = spark.createDataFrame(pandasFrame)
criapandasFrame.show(1)

criapandasFrame.toPandas()

In [ ]:
#Utilizando o RDD para criar dataframe
#passamos uma lista com tuplas para criar o dataframe

#img07
rddFrame = spark.sparkContext.parallelize([
    (1, 2., 'String 1', date(2000, 1, 1)),
    (2, 3., 'String 2', date(2000, 1, 2)),
    (3, 4., 'String 4', date(2000, 1, 3))
])

criarddFrame = spark.createDataFrame(rddFrame, schema= ['a', 'b', 'c'])
criarddFrame.show(1)
print(criarddFrame.printSchema())

criarddFrame.toPandas()

In [ ]:
#collect retorna uma lista com as linhas do dataframe
#uma Row é um registro dentro do dataframe
#todo registro dentro do dataframe é do tipo Row.

#veja o exemplo da função collect()
rangeFrame.collect()

In [ ]:
#MANIPULAÇÃO DO DATAFRAME

#CRIANDO UM NOVO DATAFRAME

produtoSchema = StructType([StructField('Produto', StringType(), False),\
                            StructField('Categoria', StringType(), False),\
                            StructField('Quantidade', IntegerType(), False),\
                            StructField('Preco', FloatType(), False)])

produtoVendas = spark.createDataFrame([['Iogurte', 'Laticinios', 4, 5.0],\
                                        ['Leite', 'Laticinios', 10, 3.5],\
                                        ['Queijo', 'Laticinios', 4, 2.5],\
                                        ['Mussarela', 'Laticinios', 5, 6.0],\
                                        ['Frango', 'Carnes', 20, 15.0],\
                                        ['Patinho', 'Carnes', 10, 20.0],\
                                        ['Carré', 'Carnes', 10, 8.0],\
                                        ['Arroz', 'Graos', 20, 3.5],\
                                        ['Feijao', 'Graos', 10, 3.5],\
                                        ['Aveia', 'Cereais', 20, 4.0],\
                                        ['Farinha trigo', 'Cereais', 5, 4.0],\
                                        ['Lentilha', 'Graos', 40, 3.5],\
                                        ['Sabao', 'Limpeza', 6, 10.0],\
                                        ['Amaciante', 'Limpeza', 5, 6.0],\
                                        ['Detergente', 'Limpeza', 20, 2.0],\
                                        ['Cloro', 'Limpeza', 5, 3.5],\
                                        ['Pera', 'Frutas', 10, 5.0],\
                                        ['Uva', 'Frutas', 6, 4.0],\
                                        ['Limao', 'Frutas', 10, 4.0],\
                                        ['Banana', 'Frutas', 6, 5.5],\
                                        ['Cenoura', 'Legumes', 20, 4.95],\
                                        ['Beterraba', 'Legumes', 40, 6.0]], produtoSchema)

print(produtoVendas.toPandas())
produtoVendas.collect()
produtoVendas.describe().show()

In [ ]:
# from pyspark.sql.functions import col, column

# col(produtoVendas)

In [ ]:
from pyspark.sql.functions import upper


#acessando colunas

#img08
# produtoVendas.select('Produto', 'Quantidade').show(5)
# produtoVendas.select(produtoVendas.Quantidade).show(5)

#img08.1

#o select permite selecionar colunas do dataframe
#além, podemos criar expressões e atribuir nomes com a função alias.
produtoVendas.select(   
                        produtoVendas.Produto,\
                        produtoVendas.Quantidade,\
                        produtoVendas.Categoria,\
                        produtoVendas.Preco,\
                        (produtoVendas.Preco + produtoVendas.Preco *  0.10).alias('Preco_10%')).\
                    show(1)
#img8.2                  
#manipulação com withColumn
#com withColumn, primeiro passa o alias da coluna e após, a expressão.
produtoVendas.withColumn('Preco_10%',\
                         (produtoVendas.Preco + produtoVendas.Preco*0.10)).\
                    show(5)

#img8.3 diferença entre withColumn e select.

#img8.4
#upper case na coluna
produtoVendas.withColumn('UpperCatg',\
                    upper('Categoria')).show(2)

#filtrando as colunas de um dataframe
#img8.5
produtoVendas.filter(produtoVendas.Categoria == "Frutas").toPandas()



In [ ]:
# produtoVendas.show()
# produtoVendas.write.csv('ProdVendas',sep= ',', header=True, mode='overwrite')

# spark.read.csv('/home/douglas/Desktop/ProjetosEstudo/Python/PySpark/ProdVendas', header= True, sep=',').show()

# spark.read.csv('../PySpark/ProdVendas', header=True, sep= ',').show()

In [ ]:
flySchema = StructType([StructField('DEST_COUNTRY_NAME', StringType(), True),\
                        StructField('ORIGIN_COUNTRY_NAME',StringType(), True),\
                        StructField('count', LongType(), False, metadata={'hello':'world'})])

flyFrame = spark.read.format('json').\
                            schema(flySchema).\
                            load('../SparkDefGuide/data/flight-data/json/2015-summary.json')

flyFrame = spark.read.\
                    schema(flySchema).\
                    json('../SparkDefGuide/data/flight-data/json/2015-summary.json')
flyFrame.show(10)